In [41]:
import pandas as pd
import numpy as np
import sys, os
import time
import io
import re
import datetime

In [51]:
### Changable parameters ###
samPath = '/Users/jacky/Documents/py3samsdk-master'
path_to_csv = 'csvs/'

year = 2012
description = '' # set a description of of this data set (e.g. flat roofs, tilted)
add_to_existing = True # if True, will add on to existing hourly.csv and yearly.csv files
############################

In [43]:
if samPath not in sys.path:
    sys.path.insert(0, samPath)
from py3samsdk.sscapi import PySSC
from urllib.error import HTTPError
from urllib.request import urlopen

ssc_lib = '/Applications/sdk-release/osx64/'  # path to SAM SSC Library
ssc = PySSC(ssc_lib)

In [44]:
zipcodes = pd.read_csv("sunroof_data/project-sunroof-postal_code-11292017.csv").loc[:,['region_name', 'state_name',  'lat_avg', 'percent_covered', 'lng_avg', 'number_of_panels_f', 'number_of_panels_total', 'yearly_sunlight_kwh_f']]
zipcodes['capacity'] = zipcodes['number_of_panels_f']*0.00025
zipcodes['region_name'] = zipcodes['region_name'].apply(lambda x: str(x)[:-2])

# Set number to threshold that we want for percent_covered
zipcodes = zipcodes[zipcodes['percent_covered'] >= 80]
zipcodes = zipcodes[zipcodes['state_name'] != 'Alaska'] # NSRDB does not have weather data for most of Alaska
latlon = zipcodes.reset_index()
latlon.head()

,index,region_name,state_name,lat_avg,percent_covered,lng_avg,number_of_panels_f,number_of_panels_total,yearly_sunlight_kwh_f,capacity
0,1,15104,Pennsylvania,40.406255,98.791687,-79.862353,42634,130282.0,1.202862e+07,10.65850
1,3,15108,Pennsylvania,40.505561,96.886620,-80.187328,397143,863308.0,1.132624e+08,99.28575
2,4,15106,Pennsylvania,40.404535,99.683730,-80.094418,133591,350858.0,3.820641e+07,33.39775
3,5,15112,Pennsylvania,40.404671,99.732083,-79.839290,12877,44229.0,3.605938e+06,3.21925
4,6,15110,Pennsylvania,40.370982,99.574633,-79.852884,38592,73782.0,1.103744e+07,9.64800


In [45]:
# info should describe the data (e.g. tilted roof, flat roof)
def create_hourly_csv(df, year, info):
    if info != '':
        info = '_' + info
    if not os.path.isfile('csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info)):
        df.to_csv('csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info))
    else:
        # append a number at the end of the filename to prevent overwritting existing files with the same name
        i = 1
        while os.path.isfile('csvs/agg_years/hourly{year}{info}({index}).csv'.format(year = year, info = info, index=i)):
            i += 1
        df.to_csv('csvs/agg_years/hourly{year}{info}({index}).csv'.format(year = year, info = info, index=i))


def create_yearly_csv(df, year, info):
    if info != '':
        info = '_' + info
    if not os.path.isfile('csvs/agg_years/yearly{year}{info}.csv'.format(year = year, info = info)):
        df.to_csv('csvs/agg_years/yearly{year}{info}.csv'.format(year = year, info = info))
    else:
        # append a number at the end of the filename to prevent overwritting existing files with the same name
        i = 1
        while os.path.isfile('csvs/agg_years/yearly{year}{info}({index}).csv'.format(year = year, info = info, index=i)):
            i += 1
        df.to_csv('csvs/agg_years/yearly{year}{info}({index}).csv'.format(year = year, info = info, index=i))

In [98]:
columns_from_sunroof = list(latlon.columns)
columns_to_add = ['generations', 'error codes', 'annual precipitation']
cloud_type_list = ['Clear', 'Probably Clear', 'Fog', 'Water', 'Super-Cooled Water', 'Mixed', 'Opaque Ice', 'Cirrus', 'Overlapping', 'Overshooting', 'Unknown']

columns = columns_from_sunroof + columns_to_add + cloud_type_list

In [106]:
yearly_df = pd.DataFrame(np.nan, index = range(len(latlon)), columns = columns)
hourly_df = pd.DataFrame(np.nan, columns = latlon['region_name'], index = pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
yearly_df = yearly_df.set_index(latlon['region_name'])

In [107]:
hourly_df

region_name,15104,15108,15106,15112,15110,15120,15116,15122,15145,15144,...,33812,33811,33810,33809,33805,33872,33853,33870,33860,33859
2012-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# yearly_generations = np.zeros(len(latlon))
# yearly_generations[:] = np.nan
hourly_generations = pd.DataFrame()
interval = '60'

# Run weather data through SAM
for index, row in latlon.iterrows():
    if not yearly.loc[row['region_name'], :].isnull().all() and not hourly.loc[row['region_name'], :].isnull().all():
        continue
    
    if os.path.isfile('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name'])):
        data = pd.read_csv('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name']))
    else:
        print("No weather data for " + row['region_name'])
        continue
    
    
    metadata = data.iloc[0:1, :]

    timezone = metadata['Time Zone'].values[0]
    elevation = metadata['Elevation'].values[0]

    # omit metadata at the top (hence the 2:)
    loc_data = data.iloc[:,:]
    loc_data.columns = loc_data.iloc[1] # reassign column names to the ones in the first row of the old table
    loc_data = loc_data.iloc[2:, :]
    loc_data = loc_data.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    loc_data = loc_data.dropna(axis = 1, how='all')
    loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']] = loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']].apply(pd.to_numeric)

    wfd = ssc.data_create()
    ssc.data_set_number(wfd, 'lat', lat)
    ssc.data_set_number(wfd, 'lon', lon)
    ssc.data_set_number(wfd, 'tz', float(timezone))
    ssc.data_set_number(wfd, 'elev', float(elevation))
    ssc.data_set_array(wfd, 'year', loc_data.index.year)
    ssc.data_set_array(wfd, 'month', loc_data.index.month)
    ssc.data_set_array(wfd, 'day', loc_data.index.day)
    ssc.data_set_array(wfd, 'hour', loc_data.index.hour)
    ssc.data_set_array(wfd, 'minute', loc_data.index.minute)
    ssc.data_set_array(wfd, 'dn', loc_data['DNI'])
    ssc.data_set_array(wfd, 'df', loc_data['DHI'])
    ssc.data_set_array(wfd, 'wspd', loc_data['Wind Speed'])
    ssc.data_set_array(wfd, 'tdry', loc_data['Temperature'])

    # Create SAM compliant object  
    dat = ssc.data_create()
    ssc.data_set_table(dat, 'solar_resource_data', wfd)
    ssc.data_free(wfd)

    # Specify the system Configuration
    # Set system capacity in MW
    system_capacity = capacity
    ssc.data_set_number(dat, 'system_capacity', system_capacity)
    # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
    ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
    # Set tilt of system in degrees
    # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
    ssc.data_set_number(dat, 'tilt', lat)
    # Set azimuth angle (in degrees) from north (0 degrees)
    ssc.data_set_number(dat, 'azimuth', 180)
    # Set the inverter efficency
    ssc.data_set_number(dat, 'inv_eff', 96)
    # Set the system losses, in percent
    ssc.data_set_number(dat, 'losses', 14.0757)
    # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
    ssc.data_set_number(dat, 'array_type', 0)
    # Set ground coverage ratio (PACKING DENSITY)
    ssc.data_set_number(dat, 'gcr', (np.cos(np.radians(lat)))**2)
    # Set constant loss adjustment
    ssc.data_set_number(dat, 'adjust:constant', 0)

    # execute and put generation results back into dataframe
    mod = ssc.module_create('pvwattsv5')
    ssc.module_exec(mod, dat)
    loc_data['generation'] = np.array(ssc.data_get_array(dat, 'gen'))
    hourly_df.loc[:, row['region_name']] = loc_data['generation'] # column of generation data from SAM model

    # free the memory
    ssc.data_free(dat)
    ssc.module_free(mod)
    print('{0}. {1}: {2}'.format(index, row['region_name'], loc_data['generation'].sum()))
    yearly_generations[index] = loc_data['generation'].sum()*1000
    yearly_precipitation[index] = loc_data['Precipitable Water'].sum() # precipitation (mm)


latlon['generations'] = yearly_generations
latlon['error codes'] = errors

KeyError: 'region_name'

In [8]:
create_hourly_csv(hourly_generations, year, description)
create_yearly_csv(latlon, year, description)

In [67]:
df = pd.read_csv("csvs/agg_years/year2012.csv")

In [68]:
year = 2012
precipitation = np.zeros(len(df))
precipitation_quarter1 = np.zeros(len(df))
precipitation_quarter2 = np.zeros(len(df))
precipitation_quarter3 = np.zeros(len(df))
precipitation_quarter4 = np.zeros(len(df))

precipitation[:], precipitation_quarter1[:], precipitation_quarter2[:], precipitation_quarter3[:], precipitation_quarter4[:] = np.nan, np.nan, np.nan, np.nan, np.nan

days_clear_sky = np.zeros(len(df))
for index, row in df.iterrows():
    if os.path.isfile('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name'])):
        data = pd.read_csv('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name']))
    else:
        print("No weather data for " + row['region_name'])
        continue
        
    loc_data = data.iloc[:,:]
    loc_data.columns = loc_data.iloc[1] # reassign column names to the ones in the first row of the old table
    loc_data = loc_data.iloc[2:, :]
    loc_data = loc_data.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    loc_data = loc_data.dropna(axis = 1, how='all')
    loc_data[['Precipitable Water', 'Cloud Type']] = loc_data[['Precipitable Water', 'Cloud Type']].apply(pd.to_numeric)
    
    precip = loc_data['Precipitable Water']
    precip_sum = precip.sum()
    quarter = 8760//4
    precip_1 = precip[:quarter].sum()
    precip_2 = precip[quarter:2*quarter].sum()
    precip_3 = precip[2*quarter: 3*quarter].sum()
    precip_4 = precip[3*quarter: 8760].sum()
    
    precipitation[index] = precip_sum
    precipitation_quarter1[index] = precip_1
    precipitation_quarter2[index] = precip_2
    precipitation_quarter3[index] = precip_3
    precipitation_quarter4[index] = precip_4
    
    print('Got precipitation for ' + str(row['region_name']))

Got precipitation for 15104
Got precipitation for 15108
Got precipitation for 15106
Got precipitation for 15112
Got precipitation for 15110
Got precipitation for 15120
Got precipitation for 15116
Got precipitation for 15122
Got precipitation for 15145
Got precipitation for 15144
Got precipitation for 15147
Got precipitation for 15146
Got precipitation for 15201
Got precipitation for 15148
Got precipitation for 15203
Got precipitation for 15202
Got precipitation for 15204
Got precipitation for 15123
Got precipitation for 15132
Got precipitation for 15131
Got precipitation for 15136
Got precipitation for 15139
Got precipitation for 15137
Got precipitation for 15142
Got precipitation for 15140
Got precipitation for 15215
Got precipitation for 15217
Got precipitation for 15216
Got precipitation for 15219
Got precipitation for 15218
Got precipitation for 15221
Got precipitation for 15220
Got precipitation for 15223
Got precipitation for 15222
Got precipitation for 15224
Got precipitation fo

Got precipitation for 18434
Got precipitation for 7010
Got precipitation for 18447
Got precipitation for 7014
Got precipitation for 18503
Got precipitation for 7016
Got precipitation for 18452
Got precipitation for 7017
Got precipitation for 18505
Got precipitation for 7018
Got precipitation for 18504
Got precipitation for 7020
Got precipitation for 17554
Got precipitation for 17568
Got precipitation for 17572
Got precipitation for 17754
Got precipitation for 17801
Got precipitation for 17831
Got precipitation for 17815
Got precipitation for 17837
Got precipitation for 17850
Got precipitation for 17579
Got precipitation for 17576
Got precipitation for 17602
Got precipitation for 17604
Got precipitation for 17603
Got precipitation for 17702
Got precipitation for 17701
Got precipitation for 6456
Got precipitation for 6457
Got precipitation for 6459
Got precipitation for 6460
Got precipitation for 6461
Got precipitation for 6468
Got precipitation for 6471
Got precipitation for 6410
Got pr

Got precipitation for 8835
Got precipitation for 8837
Got precipitation for 8840
Got precipitation for 8648
Got precipitation for 8650
Got precipitation for 8690
Got precipitation for 8691
Got precipitation for 8812
Got precipitation for 8817
Got precipitation for 8899
Got precipitation for 8901
Got precipitation for 8904
Got precipitation for 8933
Got precipitation for 10002
Got precipitation for 10003
Got precipitation for 8850
Got precipitation for 8854
Got precipitation for 8865
Got precipitation for 10566
Got precipitation for 10567
Got precipitation for 10578
Got precipitation for 10588
Got precipitation for 10589
Got precipitation for 10598
Got precipitation for 10701
Got precipitation for 10546
Got precipitation for 10547
Got precipitation for 10549
Got precipitation for 10550
Got precipitation for 10552
Got precipitation for 10553
Got precipitation for 10120
Got precipitation for 10123
Got precipitation for 10128
Got precipitation for 10154
Got precipitation for 10155
Got prec

Got precipitation for 11507
Got precipitation for 11509
Got precipitation for 11510
Got precipitation for 11514
Got precipitation for 11964
Got precipitation for 1583
Got precipitation for 11965
Got precipitation for 11967
Got precipitation for 11968
Got precipitation for 1602
Got precipitation for 11971
Got precipitation for 1603
Got precipitation for 11975
Got precipitation for 1604
Got precipitation for 11976
Got precipitation for 1605
Got precipitation for 11977
Got precipitation for 1606
Got precipitation for 11978
Got precipitation for 1607
Got precipitation for 1608
Got precipitation for 1546
Got precipitation for 11954
Got precipitation for 1562
Got precipitation for 11955
Got precipitation for 1564
Got precipitation for 11957
Got precipitation for 11958
Got precipitation for 11959
Got precipitation for 11960
Got precipitation for 1570
Got precipitation for 11962
Got precipitation for 11963
Got precipitation for 12065
Got precipitation for 12077
Got precipitation for 1745
Got p

Got precipitation for 2743
Got precipitation for 2744
Got precipitation for 2745
Got precipitation for 2747
Got precipitation for 2748
Got precipitation for 2760
Got precipitation for 2762
Got precipitation for 2763
Got precipitation for 2806
Got precipitation for 2809
Got precipitation for 2818
Got precipitation for 2824
Got precipitation for 2828
Got precipitation for 2201
Got precipitation for 2203
Got precipitation for 2210
Got precipitation for 2215
Got precipitation for 2301
Got precipitation for 2302
Got precipitation for 2322
Got precipitation for 2382
Got precipitation for 2420
Got precipitation for 2421
Got precipitation for 2445
Got precipitation for 2446
Got precipitation for 2451
Got precipitation for 2452
Got precipitation for 2453
Got precipitation for 2458
Got precipitation for 2343
Got precipitation for 2347
Got precipitation for 2351
Got precipitation for 2356
Got precipitation for 2357
Got precipitation for 2368
Got precipitation for 2370
Got precipitation for 2474
G

...

Got precipitation for 83301
Got precipitation for 94585
Got precipitation for 83401
Got precipitation for 94580
Got precipitation for 83341
Got precipitation for 94582
Got precipitation for 94578
Got precipitation for 94579
Got precipitation for 83209
Got precipitation for 94577
Got precipitation for 83202
Got precipitation for 82644
Got precipitation for 94574
Got precipitation for 94576
Got precipitation for 83201
Got precipitation for 94572
Got precipitation for 82714
Got precipitation for 94573
Got precipitation for 94568
Got precipitation for 94569
Got precipitation for 82609
Got precipitation for 94565
Got precipitation for 82604
Got precipitation for 94566
Got precipitation for 94563
Got precipitation for 94564
Got precipitation for 82009
Got precipitation for 94603
Got precipitation for 83713
Got precipitation for 83712
Got precipitation for 94613
Got precipitation for 83706
Got precipitation for 94611
Got precipitation for 83705
Got precipitation for 94612
Got precipitation fo

Got precipitation for 44614
Got precipitation for 32937
Got precipitation for 32935
Got precipitation for 32934
Got precipitation for 32931
Got precipitation for 27110
Got precipitation for 27109
Got precipitation for 27127
Got precipitation for 27235
Got precipitation for 33122
Got precipitation for 33109
Got precipitation for 33101
Got precipitation for 27051
Got precipitation for 27101
Got precipitation for 27104
Got precipitation for 27103
Got precipitation for 27106
Got precipitation for 27105
Got precipitation for 27107
Got precipitation for 33022
Got precipitation for 33013
Got precipitation for 33010
Got precipitation for 33020
Got precipitation for 33019
Got precipitation for 60463
Got precipitation for 60464
Got precipitation for 45341
Got precipitation for 60458
Got precipitation for 45322
Got precipitation for 60459
Got precipitation for 60461
Got precipitation for 45324
Got precipitation for 60462
Got precipitation for 60466
Got precipitation for 45344
Got precipitation fo

Got precipitation for 85016
Got precipitation for 85009
Got precipitation for 85008
Got precipitation for 95319
Got precipitation for 85013
Got precipitation for 95301
Got precipitation for 85012
Got precipitation for 95307
Got precipitation for 85045
Got precipitation for 85044
Got precipitation for 85050
Got precipitation for 85048
Got precipitation for 85043
Got precipitation for 60436
Got precipitation for 60438
Got precipitation for 60439
Got precipitation for 60431
Got precipitation for 60432
Got precipitation for 60433
Got precipitation for 60435
Got precipitation for 60441
Got precipitation for 60443
Got precipitation for 85053
Got precipitation for 85051
Got precipitation for 85083
Got precipitation for 85054
Got precipitation for 85031
Got precipitation for 85029
Got precipitation for 85033
Got precipitation for 85032
Got precipitation for 60428
Got precipitation for 60422
Got precipitation for 60423
Got precipitation for 60425
Got precipitation for 60426
Got precipitation fo

Got precipitation for 33904
Got precipitation for 46234
Got precipitation for 33903
Got precipitation for 33901
Got precipitation for 46227
Got precipitation for 46228
Got precipitation for 33907
Got precipitation for 46229
Got precipitation for 33905
Got precipitation for 46231
Got precipitation for 28412
Got precipitation for 46237
Got precipitation for 28411
Got precipitation for 46239
Got precipitation for 46240
Got precipitation for 46241
Got precipitation for 33914
Got precipitation for 28428
Got precipitation for 33913
Got precipitation for 33912
Got precipitation for 46235
Got precipitation for 28449
Got precipitation for 33909
Got precipitation for 46236
Got precipitation for 33971
Got precipitation for 33967
Got precipitation for 33966
Got precipitation for 46342
Got precipitation for 33976
Got precipitation for 46383
Got precipitation for 33974
Got precipitation for 33973
Got precipitation for 28637
Got precipitation for 33972
Got precipitation for 46394
Got precipitation fo

Got precipitation for 28277
Got precipitation for 33781
Got precipitation for 28274
Got precipitation for 33778
Got precipitation for 46107
Got precipitation for 33803
Got precipitation for 33801
Got precipitation for 33786
Got precipitation for 46112
Got precipitation for 46075
Got precipitation for 46077
Got precipitation for 33777
Got precipitation for 33776
Got precipitation for 33774
Got precipitation for 28209
Got precipitation for 33760
Got precipitation for 28211
Got precipitation for 28210
Got precipitation for 28213
Got precipitation for 28212
Got precipitation for 28215
Got precipitation for 28214
Got precipitation for 33765
Got precipitation for 28217
Got precipitation for 33764
Got precipitation for 28216
Got precipitation for 33763
Got precipitation for 33773
Got precipitation for 28223
Got precipitation for 33772
Got precipitation for 46055
Got precipitation for 33771
Got precipitation for 33770
Got precipitation for 46060
Got precipitation for 46038
Got precipitation fo

In [69]:
df['Annual Precipation'] = precipitation
df['Quarter 1 Precipitation'] = precipitation_quarter1
df['Quarter 2 Precipitation'] = precipitation_quarter2
df['Quarter 3 Precipitation'] = precipitation_quarter3
df['Quarter 4 Precipitation'] = precipitation_quarter4

In [70]:
df.columns

Index(['Unnamed: 0', 'index', 'region_name', 'state_name', 'lat_avg',
       'percent_covered', 'lng_avg', 'number_of_panels_f',
       'number_of_panels_total', 'yearly_sunlight_kwh_f', 'capacity',
       'generations', 'error codes', 'Annual Precipation',
       'Quarter 1 Precipitation', 'Quarter 2 Precipitation',
       'Quarter 3 Precipitation', 'Quarter 4 Precipitation'],
      dtype='object')

In [71]:
create_yearly_csv(df, year, 'tilted')

In [78]:
cloud_type_list = ['region_name', 'Clear', 'Probably Clear', 'Fog', 'Water', 'Super-Cooled Water', 'Mixed', 'Opaque Ice', 'Cirrus', 'Overlapping', 'Overshooting', 'Unknown']

In [79]:
len(cloud_type_list)

11

In [93]:
year = 2012
# precipitation = np.zeros(len(df))
# precipitation_quarter1 = np.zeros(len(df))
# precipitation_quarter2 = np.zeros(len(df))
# precipitation_quarter3 = np.zeros(len(df))
# precipitation_quarter4 = np.zeros(len(df))
clouds = pd.DataFrame(np.nan, index = range(8760), columns = cloud_type_list)

# precipitation[:], precipitation_quarter1[:], precipitation_quarter2[:], precipitation_quarter3[:], precipitation_quarter4[:] = np.nan, np.nan, np.nan, np.nan, np.nan

days_clear_sky = np.zeros(len(df))
for index, row in df.iterrows():
#     print("Collected cloud data for row " + str(index))
    clouds.loc[index, 'region_name'] = row['region_name']
    if os.path.isfile('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name'])):
        data = pd.read_csv('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name']))
    else:
        print("No weather data for " + row['region_name'])
        continue
        
    loc_data = data.iloc[:,:]
    loc_data.columns = loc_data.iloc[1] # reassign column names to the ones in the first row of the old table
    loc_data = loc_data.iloc[2:, :]
    loc_data = loc_data.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    loc_data = loc_data.dropna(axis = 1, how='all')
    loc_data[['Precipitable Water', 'Cloud Type']] = loc_data[['Precipitable Water', 'Cloud Type']].apply(pd.to_numeric)
    
    unique, counts = np.unique(loc_data['Cloud Type'], return_counts=True)
    counts = dict(zip(unique, counts))
    for cloud_num in counts.keys():
        if cloud_num < 0:
            cloud_type = 'Unknown'
        else:
            cloud_type = cloud_type_list[cloud_num]
        clouds.loc[index, cloud_type] = counts[cloud_num]
    sys.stdout.write("\rGot %i\n" % (index))
    sys.stdout.flush()
    
    

Got 4541No weather data for 99501
No weather data for 99502
Got 4548No weather data for 99506
No weather data for 99507
No weather data for 99503
No weather data for 99504
No weather data for 99258
Got 4571No weather data for 99517
No weather data for 99518
No weather data for 99515
No weather data for 99509
No weather data for 99513
No weather data for 99508
Got 4588No weather data for 99701
No weather data for 99530
No weather data for 99599
Got 8022

In [112]:
create_hourly_csv(clouds, 2012, 'clouds')